# imports & functions

In [5]:
import os
import shutil
import pandas as pd

def list_all_datapaths(data_path):
    data_paths = []
    for file in os.listdir(data_path):
        if 'EJT' in file[0:5] or 'revision' in data_path:
            for file_ in os.listdir(data_path+file):
                if 'record' in file_:
                    data_paths += [data_path+file+'\\'+file_]
            
    return data_paths

def copy_files(source_file, dest_file):
    if not os.path.exists(dest_file):
        shutil.copy(source_file, dest_file)
        print(f"Copied {source_file} to {dest_file}")
    else:
        print(f"File already exists at {dest_file}, not copied.")
        


# main

In [6]:
# get current working directory
current_dir = os.getcwd()
recordings_date_data = pd.read_csv(os.path.join(current_dir,'consecutive_data.csv'))

out_path = r"Z:\projects\sequence_squad\revision_data\emmett_revisions\sleep_wake_link_data\replay_to_behaviour\\"

In [ ]:
for i in range(len(recordings_date_data)):

    current_row = recordings_date_data.loc[recordings_date_data.index[i]]
    
    # Create destination directory if it doesn't exist
    destination_folder1 = os.path.join(out_path,current_row['mouse_id'],'day1_' + current_row['behaviour 1'].replace('/','-'))
    os.makedirs(destination_folder1, exist_ok=True)
    
    if 'EJT' in current_row['mouse_id']:
        organised_path = r"Z:\projects\sequence_squad\organised_data\animals\\"
        row_mouse = current_row['mouse_id'].split('T')[-1]
    else:
        organised_path = r"Z:\projects\sequence_squad\revision_data\organised_data\animals\\"
        row_mouse = current_row['mouse_id'].lower()
        if 'ap5' in row_mouse:
            row_mouse = row_mouse.replace('r','R')

    ##############################
    # if not 'ap5' in row_mouse:
    #     continue  # skip if not AP5 mouse
    ##############################

    # load in the first days data:
    
    # list out all possible datapaths
    data_paths = list_all_datapaths(organised_path) 

    for path in data_paths:
        if 'EJT' in path:
            a = path.split('\\')[-2].split('_')[0][3::]
        else:
            a = path.split('\\')[-2].split('_')[0]
        b = path.split('\\')[-2][-1]
        c = path.split('\\')[-1].split('_')[0].split('g')[-1]
        current_mouse = '_'.join([a,b,c])

        if current_mouse != row_mouse:
            continue
        print('-------------------------------')
        print(f"behav 1 =  {current_row['mouse_id']} {current_row['behaviour 1']}")
    
        for stage in os.listdir(os.path.join(path,'behav_sync')):
            if 'task' in stage:
                behav1_path = os.path.join(path,'behav_sync',stage)
                # Copy file only if it doesn't already exist
                copy_files(os.path.join(behav1_path,'Behav_Ephys_Camera_Sync.csv'), os.path.join(destination_folder1,'Behav_Ephys_Camera_Sync.csv'))
                
        # get tracking data for behaviour 1
        video_path = os.path.join(path,'video','tracking') + r'\\'
        if os.path.isdir(video_path) == False:
            print('Cant find video:', video_path)

        if 'EJT' in current_row['mouse_id']:
            for stage in os.listdir(video_path):
                if 'task' in stage: 
                    tracking_path = video_path + stage + '\\'
        else:
            tracking_path = video_path 
                    
        for track_file in os.listdir(tracking_path):
            if 'back' in track_file and 'task' in track_file and '.h5' in track_file:
                print(f'Found task tracking file {track_file}')
                copy_files(os.path.join(tracking_path,track_file), os.path.join(destination_folder1,track_file))
            if 'back' in track_file and 'port' in track_file and '.h5' in track_file:
                print(f'Found port tracking file {track_file}')
                copy_files(os.path.join(tracking_path,track_file), os.path.join(destination_folder1,track_file))
            if 'BACK' in track_file and not 'PORT' in track_file and '.h5' in track_file:
                print(f'Found task tracking file {track_file}')
                copy_files(os.path.join(tracking_path,track_file), os.path.join(destination_folder1,track_file))
            if 'BACK' in track_file and 'PORT' in track_file and '.h5' in track_file:
                print(f'Found port tracking file {track_file}')
                copy_files(os.path.join(tracking_path,track_file), os.path.join(destination_folder1,track_file))
    
    
    ################## try to data for behaviour 2
    
    # Create destination directory if it doesn't exist
    destination_folder2 = os.path.join(out_path,current_row['mouse_id'],'day2_' + current_row['behaviour 2'].replace('/','-'))
    os.makedirs(destination_folder2, exist_ok=True)
    
    # search in each possible location for the data
    File_found = False
    copied = False
    # 1. organised data structures
    for path_ in data_paths:
        path_date = path_.split('_')[-1].replace('-', '/')
        if current_row['behaviour 2'] == path_date:
            path_mouse_implant_code = row_mouse.split('_')[0] + '_implant' + row_mouse.split('_')[1]
            if path_mouse_implant_code in path_:
                File_found = True            
        if not File_found:
            continue
        # reset the file found flag
        File_found = False
        print('-------------------------------')
        print(f"behav 2 =  {current_row['mouse_id']} {current_row['behaviour 2']}")
        print(f"path = {path_}")

        # copy over the behav data: 
        for stage in os.listdir(os.path.join(path_,'behav_sync')):
            if 'task' in stage:
                behav2_path = os.path.join(path_,'behav_sync',stage)
                # Copy file only if it doesn't already exist
                copy_files(os.path.join(behav2_path,'Behav_Ephys_Camera_Sync.csv'), os.path.join(destination_folder2,'Behav_Ephys_Camera_Sync.csv')) 
                
        ## find camera data for behaviour 2
        video_path = os.path.join(path_,'video','tracking') + r'\\'
        if os.path.isdir(video_path) == False:
            print('Cant find video:', video_path)

        if 'EJT' in current_row['mouse_id']:
            for stage in os.listdir(video_path):
                if 'task' in stage: 
                    tracking_path = video_path + stage + '\\'
        else:
            tracking_path = video_path
                    
        for track_file in os.listdir(tracking_path):
            if 'back' in track_file and 'task' in track_file and '.h5' in track_file:
                print(f'Found task tracking file {track_file}')
                copy_files(os.path.join(tracking_path,track_file), os.path.join(destination_folder2,track_file))
            if 'back' in track_file and 'port' in track_file and '.h5' in track_file:
                print(f'Found port tracking file {track_file}')
                copy_files(os.path.join(tracking_path,track_file), os.path.join(destination_folder2,track_file))
            if 'BACK' in track_file and not 'PORT' in track_file and '.h5' in track_file:
                print(f'Found task tracking file {track_file}')
                copy_files(os.path.join(tracking_path,track_file), os.path.join(destination_folder2,track_file))
            if 'BACK' in track_file and 'PORT' in track_file and '.h5' in track_file:
                print(f'Found port tracking file {track_file}')
                copy_files(os.path.join(tracking_path,track_file), os.path.join(destination_folder2,track_file))
                # processes these one by one and put them in a folder
        copied = True
                    
                    
    if not copied:
        print('NOPE! bpod data (and camera data?) needs to be processed!')


            
        

-------------------------------
behav 1 =  AP5R_1_1 16/11/2024
Copied Z:\projects\sequence_squad\revision_data\organised_data\animals\\ap5R_implant1\recording1_16-11-2024\behav_sync\2_task\Behav_Ephys_Camera_Sync.csv to Z:\projects\sequence_squad\revision_data\emmett_revisions\sleep_wake_link_data\replay_to_behaviour\\AP5R_1_1\day1_16-11-2024\Behav_Ephys_Camera_Sync.csv
Found port tracking file back_2023-05-14T12_23_41DLC_resnet50_port-tracking_back_viewMay2shuffle1_500000.h5
Copied Z:\projects\sequence_squad\organised_data\animals\\EJT270_implant1\recording3_14-05-2023\video\tracking\\2_task\back_2023-05-14T12_23_41DLC_resnet50_port-tracking_back_viewMay2shuffle1_500000.h5 to Z:\projects\sequence_squad\revision_data\emmett_revisions\sleep_wake_link_data\replay_to_behaviour\\AP5R_1_1\day1_16-11-2024\back_2023-05-14T12_23_41DLC_resnet50_port-tracking_back_viewMay2shuffle1_500000.h5
Found task tracking file back_2023-05-14T12_23_41DLC_resnet50_Yellow-floor-task-tracking-backviewApr9shuff

In [ ]:
# find the behavioural data from the day of the recording and the day after

# if both recordings have tracking data then extract this also

# extract the features of behaviour as i did before

C:\Users\Emmett Thompson\Documents\sequences_analyse_PPseq\revision_analysis\prioritisation_analysis\sleep_behaviour_link\consecutive_data.csv